In [ ]:
import cv2 as cv
from pathlib import Path

def load_img(path):
    return cv.imread(str(path.resolve()), cv.IMREAD_GRAYSCALE)
    
def convert2binary(img):
    _, binary = cv.threshold(img, 127, 1, cv.THRESH_BINARY_INV)
    return binary

def small_slice(img):
    x = 0
    y = 0
    w = 100
    h = 100
    return img[y:y+h, x:x+w]

def find_corner(img):
    # first point
    # find y
    y_index = 0
    for y in img:
        if any(y):
            break
        y_index += 1
        
    # find x
    x_index = 0
    for x in img[y_index]:
        if x == 1:
            break
        x_index += 1
    
    #second point
    break_first_loop = False
    sec_x_index = 0
    for x in img[0]:
        if not break_first_loop:
            sec_y_index = 0
            for y in img:
                if y[sec_x_index] == 1:
                    break_first_loop = True
                    break
                sec_y_index += 1
        else:
            break
        sec_x_index += 1
        
    # determine CW or CCW
    if (x_index - sec_x_index) > (sec_y_index - y_index):
        # second point, CCW
#         print(f"1st: {x_index}, {y_index} | 2nd: {sec_x_index}, {sec_y_index} | winner: 2")
        return sec_x_index, sec_y_index
    elif (x_index - sec_x_index) == 0:
        # first point, both Xs are the same
#         print(f"1st: {x_index}, {y_index} | 2nd: {sec_x_index}, {sec_y_index} | winner: 1")
        return x_index, y_index
    elif (sec_y_index - y_index) == 0:
        # second point, both Ys are the same
#         print(f"1st: {x_index}, {y_index} | 2nd: {sec_x_index}, {sec_y_index} | winner: 2")
        return sec_x_index, sec_y_index
    else:            
        # first point, CW
#         print(f"1st: {x_index}, {y_index} | 2nd: {sec_x_index}, {sec_y_index} | winner: 1")
        return x_index, y_index

def mark_corner(img, center):
    small_radius = 2
    big_radius = 6
    color = (0,0,0) #black

    circled = cv.circle(img, center, small_radius, color)
    circled = cv.circle(img, center, big_radius, color)
    return circled

In [ ]:
for f in Path("deskewed/").iterdir():
    file_ = str(f.resolve())
    if not str(f.name).startswith(".") and str(f.name).endswith(".jpg"):
        original = load_img(f)
        binary = convert2binary(original)
        
        binary_sliced = small_slice(binary)
        original_sliced = small_slice(original)
        
        # label the corner
        x, y = find_corner(binary_sliced)
        center = (x, y)
        circled = mark_corner(original_sliced, center)
        
        # show marked corner for manual decision
        cv.imshow("circled", circled)
        k = cv.waitKey(0)
        if k == 27: #esc
            cv.destroyAllWindows()
            exit()
            break

        # decide to keep or reject
        if k == ord("a"):
#             crop original and save
            h = original.shape[0]
            w = original.shape[1]
            cropped = original[y:y+h, x:x+w]
            cv.imwrite(f"fine_cropped/{str(f.name)}", cropped)
        if k == ord("r"):
#             reject, save original to need_to_fine_crop
            cv.imwrite(f"need_to_fine_crop/{str(f.name)}", original)

